In [1]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

/home/jallpa/GitHub/object-detection/env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
train_data, validation_data, test_data = object_detector.DataLoader.from_csv('gs://cloud-ml-data/img/openimage/csv/salads_ml_use.csv')


2022-06-08 09:11:20.402931: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


In [5]:
spec = model_spec.get('efficientdet_lite0')

2022-06-08 09:15:18.372422: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-08 09:15:18.373112: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-08 09:15:18.377509: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-08 09:15:18.377970: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-08 09:15:18.378607: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

In [8]:
model = object_detector.create(train_data,
                               model_spec=spec,
                               batch_size=8,
                               train_whole_model=True,
                               validation_data=validation_data,
                               epochs = 10
)

Epoch 1/10
21/21 [==============================] - 34s 627ms/step - det_loss: 1.7497 - cls_loss: 1.1283 - box_loss: 0.0124 - reg_l2_loss: 0.0635 - loss: 1.8132 - learning_rate: 0.0090 - gradient_norm: 0.7807 - val_det_loss: 1.6277 - val_cls_loss: 1.0807 - val_box_loss: 0.0109 - val_reg_l2_loss: 0.0635 - val_loss: 1.6912
Epoch 2/10
21/21 [==============================] - 6s 272ms/step - det_loss: 1.5750 - cls_loss: 1.0535 - box_loss: 0.0104 - reg_l2_loss: 0.0635 - loss: 1.6385 - learning_rate: 0.0093 - gradient_norm: 1.0722 - val_det_loss: 1.3913 - val_cls_loss: 0.8912 - val_box_loss: 0.0100 - val_reg_l2_loss: 0.0635 - val_loss: 1.4547
Epoch 3/10
21/21 [==============================] - 8s 383ms/step - det_loss: 1.3906 - cls_loss: 0.9112 - box_loss: 0.0096 - reg_l2_loss: 0.0635 - loss: 1.4541 - learning_rate: 0.0082 - gradient_norm: 1.7547 - val_det_loss: 3.1117 - val_cls_loss: 2.6433 - val_box_loss: 0.0094 - val_reg_l2_loss: 0.0635 - val_loss: 3.1752
Epoch 4/10
21/21 [===============

In [9]:
model.evaluate(test_data)

1/1 [==============================] - 6s 6s/step



{'AP': 0.1183255,
 'AP50': 0.22658026,
 'AP75': 0.11546995,
 'APs': -1.0,
 'APm': 0.21176508,
 'APl': 0.118041255,
 'ARmax1': 0.08518382,
 'ARmax10': 0.1950312,
 'ARmax100': 0.24193293,
 'ARs': -1.0,
 'ARm': 0.5833333,
 'ARl': 0.23950303,
 'AP_/Baked Goods': 0.0,
 'AP_/Salad': 0.38840255,
 'AP_/Cheese': 0.011169391,
 'AP_/Seafood': 0.0,
 'AP_/Tomato': 0.19205555}

In [21]:
model.export(export_dir='.')

2022-06-08 10:55:57.255721: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-06-08 10:56:15.516387: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.
2022-06-08 10:56:25.706437: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-06-08 10:56:25.706461: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-06-08 10:56:25.707335: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpvgacxjbx
2022-06-08 10:56:25.784601: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-06-08 10:56:25.784629: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if pre